<a href="https://colab.research.google.com/github/AI-Lab-2025-2-3rd/ai-project-myth/blob/main/superconductor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. GitHub에서 CSV 불러오기
import pandas as pd

# GitHub에 올린 raw 파일 경로 입력
url = "https://raw.githubusercontent.com/AI-Lab-2025-2-3rd/ai-project-myth/refs/heads/main/train.csv"

# 2. Pandas로 읽기
df = pd.read_csv(url)

# 3. 데이터 확인
print(df.shape)
print(df.head())


(12759, 83)
            ID  number_of_elements  mean_atomic_mass  wtd_mean_atomic_mass  \
0  TRAIN_00000                   3           0.42099               0.39479   
1  TRAIN_00001                   4           0.34401               0.22868   
2  TRAIN_00002                   4           0.34437               0.25134   
3  TRAIN_00003                   2           0.43715               0.45117   
4  TRAIN_00004                   4           0.18946               0.19625   

   gmean_atomic_mass  wtd_gmean_atomic_mass  entropy_atomic_mass  \
0            0.42310                0.40580              0.55107   
1            0.26533                0.16498              0.60467   
2            0.26510                0.16350              0.60353   
3            0.44012                0.46167              0.34778   
4            0.16067                0.17351              0.62604   

   wtd_entropy_atomic_mass  range_atomic_mass  wtd_range_atomic_mass  ...  \
0                  0.41532       